# Movie Maps
---
Webscrapper for Movie data.
---
### Dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from os.path import basename
from pprint import pprint
import time
from datetime import datetime as dt
import pandas as pd
import numpy as np

In [2]:
# Splinter set up
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

---
### Extraction
- Scrape data off of Box Office Mojo

In [3]:
def scrapeMovies(id_url, data_url):
    browser.visit(id_url)
    # Direct browser to page
    html = browser.html
    bom_soup = BeautifulSoup(html, 'html.parser')

    # Focus on 'body' content
    main_body = bom_soup.find('div', id='body')

    # Focus on the 4th table in body
    main_table = main_body.findAll('table')[3]
    
    # Extract the rows associated with movie links
    white_rows = main_table.findAll("tr", bgcolor="#ffffff")
    bluey_rows = main_table.findAll("tr", bgcolor="#f4f4ff")

    # Extract movie IDs from the links
    movie_ids = []
    movie_ttl = []

    for row in white_rows:
        link = row.find('a')
        if (link):
            addr = link['href'].split('=')[1].split('.')[0]
            movie_ids.append(addr)
            movie_ttl.append(link.get_text())
    for row in bluey_rows:
        link = row.find('a')
        if (link):
            addr = link['href'].split('=')[1].split('.')[0]
            movie_ids.append(addr)
            movie_ttl.append(link.get_text())

    # Scrape the weekly gross data on the targeted movies
    movie_pds = []
    counter = 0
    for ids in movie_ids:
        time.sleep(1)
        #print(f"{counter}: {ids};")
        wk_html = pd.read_html(data_url + ids + '.htm')
        if (wk_html):
            # slice column
            wk_col = wk_html[0].iloc[10, :9]

            # slice data
            wk_pd = wk_html[0].iloc[11:, :9]

            # fix column labels
            wk_col[6:9] = wk_col[5:8]
            wk_col[0] = 'Date'
            wk_col[4:6] = ['Theaters', 'Change']

            # set column labels
            wk_pd.columns = wk_col
            
            wk_pd.reset_index(inplace=True)

            movie_pds.append(wk_pd)
        counter = counter + 1
    # Return findings
    return {"titles":movie_ttl, "ids":movie_ids, "pds":movie_pds}

In [4]:
# Direct browser to the Box Office Mojo website and prepare for scraping
base_url = 'https://www.boxofficemojo.com' 
url = base_url + '/yearly/chart/?yr=2018&p=.htm'
browser.visit(url)

In [5]:
# Direct browser to page
html = browser.html
bom_soup = BeautifulSoup(html, 'html.parser')

# Focus on 'body' content
main_body = bom_soup.find('div', id='body')

# Focus on the 4th table in body
main_table = main_body.findAll('table')[3]

# Inspect html
print(main_table)


<table border="0" cellpadding="0" cellspacing="0" width="100%"><tbody><tr><td align="center" valign="top"><br/>
<h1>2018 DOMESTIC GROSSES</h1><p>Total Grosses of all Movies Released in 2018</p>
<center><font face="Verdana" size="4"><b><font face="Verdana" size="5">#1–100</font> - <a href="/yearly/chart/?page=2&amp;view=releasedate&amp;view2=domestic&amp;yr=2018&amp;p=.htm">#101–200</a> - <a href="/yearly/chart/?page=3&amp;view=releasedate&amp;view2=domestic&amp;yr=2018&amp;p=.htm">#201–300</a> - <a href="/yearly/chart/?page=4&amp;view=releasedate&amp;view2=domestic&amp;yr=2018&amp;p=.htm">#301–400</a> - <a href="/yearly/chart/?page=5&amp;view=releasedate&amp;view2=domestic&amp;yr=2018&amp;p=.htm">#401–500</a> - <a href="/yearly/chart/?page=6&amp;view=releasedate&amp;view2=domestic&amp;yr=2018&amp;p=.htm">#501–600</a> - <a href="/yearly/chart/?page=7&amp;view=releasedate&amp;view2=domestic&amp;yr=2018&amp;p=.htm">#601–700</a> - <a href="/yearly/chart/?page=8&amp;view=releasedate&amp;vie

In [6]:
# Extract the rows associated with movie links
white_rows = main_table.findAll("tr", bgcolor="#ffffff")
bluey_rows = main_table.findAll("tr", bgcolor="#f4f4ff")

# Examine row structure
print(white_rows[0])

<tr bgcolor="#ffffff"><td align="center"><font size="2">1</font></td><td><b><font size="2"><a href="/movies/?id=marvel2017b.htm">Black Panther</a></font></b></td><td><font size="2"><a href="/studio/chart/?studio=buenavista.htm">BV</a></font></td><td align="right"><font size="2"><b>$700,059,566</b></font></td><td align="right"><font size="2">4,084</font></td><td align="right"><font size="2">$202,003,951</font></td><td align="right"><font size="2">4,020</font></td><td align="right"><font size="2"><a href="/schedule/?view=bydate&amp;release=theatrical&amp;date=2018-02-16&amp;p=.htm">2/16</a></font></td><td align="right"><font size="2">8/9</font></td></tr>


In [7]:
# Extract movie IDs from the links
movie_ids = []
movie_ttl = []

for row in white_rows:
    link = row.find('a')
    if (link):
        addr = link['href'].split('=')[1].split('.')[0]
        movie_ids.append(addr)
        movie_ttl.append(link.get_text())
for row in bluey_rows:
    link = row.find('a')
    if (link):
        addr = link['href'].split('=')[1].split('.')[0]
        movie_ids.append(addr)
        movie_ttl.append(link.get_text())

# Check extraction results
print(len(movie_ids))
for i in range(5):
    print(f"{movie_ttl[i]}: {movie_ids[i]}")

100
Black Panther: marvel2017b
Incredibles 2: theincredibles2
Deadpool 2: foxmarvel18
Mission: Impossible - Fallout: missionimpossible6
Solo: A Star Wars Story: untitledhansolostarwarsanthologyfilm


In [8]:
# Determine how to scrape movie data from webpage 
wk_html = pd.read_html('https://www.boxofficemojo.com/movies/?page=weekly&id=marvel2017b.htm')

# slice column
wk_col = wk_html[0].iloc[10, 0:9]

# slice data
wk_pd = wk_html[0].iloc[11:, 0:9]

# fix column labels
wk_col[6:9] = wk_col[5:8]
wk_col[0] = 'Date'
wk_col[4:6] = ['Theaters', 'Change']

# set column labels
wk_pd.columns = wk_col

# preview results
wk_pd.head()

10,Date,Rank,WeeklyGross,%Change,Theaters,Change,Avg.,Gross-to-Date,Week#
11,Feb 1622,1,"$291,954,422",-,4020,-,"$72,625","$291,954,422",1
12,Feb 23Mar 1,1,"$143,445,615",-50.9%,4020,-,"$35,683","$435,400,037",2
13,Mar 28,1,"$85,479,564",-40.4%,4084,+64,"$20,930","$520,879,601",3
14,Mar 915,1,"$57,496,927",-32.7%,3942,-142,"$14,586","$578,376,528",4
15,Mar 1622,1,"$35,881,708",-37.6%,3834,-108,"$9,359","$614,258,236",5


In [9]:
# Base url for targeted scrape data
wk_base_url = 'https://www.boxofficemojo.com/movies/?page=weekly&id='


In [10]:
movie_pds = []
counter = 0
for ids in movie_ids:
#     if((counter > 0) & (counter%20 == 0)):
    time.sleep(1)
    print(f"{counter}: {ids};")
    wk_html = pd.read_html(wk_base_url + ids + '.htm')
    if (wk_html):
        # slice column
        wk_col = wk_html[0].iloc[10, :9]

        # slice data
        wk_pd = wk_html[0].iloc[11:, :9]

        # fix column labels
        wk_col[6:9] = wk_col[5:8]
        wk_col[0] = 'Date'
        wk_col[4:6] = ['Theaters', 'Change']

        # set column labels
        wk_pd.columns = wk_col

        movie_pds.append(wk_pd)
    counter = counter + 1
    

print(f'"{movie_ttl[1]}": {movie_ids[1]}')
movie_pds[1].head()

0: marvel2017b;
1: theincredibles2;
2: foxmarvel18;
3: missionimpossible6;
4: untitledhansolostarwarsanthologyfilm;
5: astarisborn2018;
6: bohemianrhapsody;
7: hoteltransylvania3;
8: wbevent2018;
9: wbeventfilm2018;
10: readyplayerone;
11: thenun;
12: theequalizer2;
13: fiftyshadesfreed;
14: disneyfairytale2017;
15: icanonlyimagine;
16: nightschool2018;
17: gamenight;
18: thehousewithaclockinitswalls;
19: insidious4;
20: pacificrim2;
21: dcfilm0318;
22: tag;
23: nutcrackerandthefourrealms;
24: thepredator;
25: sicario2;
26: blackkklansman;
27: goosebumps2;
28: horsesoldiers2018;
29: firstman;
30: acrimony;
31: uncledrew;
32: untitledgregberlantifilm;
33: widows;
34: 1517toparis;
35: sonyeventfilm2017;
36: deathwish2017;
37: annihilation;
38: untitledtylerperrymovie;
39: supertroopers2;
40: wbanimation62018;
41: robinhood2018;
42: winchester;
43: whiteboyrick;
44: overlord;
45: proudmary;
46: superfly;
47: badtimesattheelroyale;
48: paulapostleofchrist;
49: chappaquiddick;
50: marvel051

10,Date,Rank,WeeklyGross,%Change,Theaters,Change,Avg.,Gross-to-Date,Week#
11,Jun 1521,1,"$269,446,690",-,4410,-,"$61,099","$269,446,690",1
12,Jun 2228,2,"$124,736,824",-53.7%,4410,-,"$28,285","$394,183,514",2
13,Jun 29Jul 5,2,"$81,177,900",-34.9%,4410,-,"$18,408","$475,361,414",3
14,Jul 612,2,"$44,237,078",-45.5%,4113,-297,"$10,755","$519,598,492",4
15,Jul 1319,4,"$26,216,948",-40.7%,3705,-408,"$7,076","$545,815,440",5


#### Function Based Scraping

In [11]:
# Common base url
base_url = 'https://www.boxofficemojo.com' 

# Category URL's for scraping:
y2018_url = '/yearly/chart/?yr=2018&p=.htm'
y2017_url = '/yearly/chart/?yr=2017&p=.htm'


# Data URL's for scraping:
wkly_url = '/movies/?page=weekly&id='

In [12]:
res2018 = scrapeMovies(base_url+y2018_url, base_url+wkly_url)

In [13]:
res2018['pds'][0]

10,index,Date,Rank,WeeklyGross,%Change,Theaters,Change,Avg.,Gross-to-Date,Week#
0,11,Feb 1622,1,"$291,954,422",-,4020,-,"$72,625","$291,954,422",1
1,12,Feb 23Mar 1,1,"$143,445,615",-50.9%,4020,-,"$35,683","$435,400,037",2
2,13,Mar 28,1,"$85,479,564",-40.4%,4084,+64,"$20,930","$520,879,601",3
3,14,Mar 915,1,"$57,496,927",-32.7%,3942,-142,"$14,586","$578,376,528",4
4,15,Mar 1622,1,"$35,881,708",-37.6%,3834,-108,"$9,359","$614,258,236",5
5,16,Mar 2329,2,"$25,178,398",-29.8%,3370,-464,"$7,471","$639,436,634",6
6,17,Mar 30Apr 5,3,"$17,489,106",-30.5%,2988,-382,"$5,853","$656,925,740",7
7,18,Apr 612,4,"$11,529,782",-34.1%,2747,-241,"$4,197","$668,455,522",8
8,19,Apr 1319,6,"$7,986,587",-30.7%,2180,-567,"$3,664","$676,442,109",9
9,20,Apr 2026,8,"$7,186,380",-10.0%,1930,-250,"$3,724","$683,628,489",10


---
### Transformation
- Convert PDs to a csv format for chord diagram

In [14]:
dates = res2018['pds'][0]['Date']
dates
for date in dates:
    p_date = date.split('')[0].split(' ')
    if (len(p_date[1])==1):
        p_date[1] = '0'+ p_date[1]
    start_str = "-".join(p_date)+'-18'
    start_date = dt.strptime(start_str, '%b-%d-%y')
    week_nm = start_date.strftime('%U')
    print(f"{start_str} = week # {week_nm}")

Feb-16-18 = week # 06
Feb-23-18 = week # 07
Mar-02-18 = week # 08
Mar-09-18 = week # 09
Mar-16-18 = week # 10
Mar-23-18 = week # 11
Mar-30-18 = week # 12
Apr-06-18 = week # 13
Apr-13-18 = week # 14
Apr-20-18 = week # 15
Apr-27-18 = week # 16
May-04-18 = week # 17
May-11-18 = week # 18
May-18-18 = week # 19
May-25-18 = week # 20
Jun-01-18 = week # 21
Jun-08-18 = week # 22
Jun-15-18 = week # 23
Jun-22-18 = week # 24
Jun-29-18 = week # 25
Jul-06-18 = week # 26
Jul-13-18 = week # 27
Jul-20-18 = week # 28
Jul-27-18 = week # 29
Aug-03-18 = week # 30


In [15]:
def getWkNum(datestring, yr):
    p_date = datestring.split('')[0].split(' ')
    if (len(p_date[1])==1):
        p_date[1] = '0'+ p_date[1]
    start_str = '-'.join(p_date) + f'-{yr}'
    start_date = dt.strptime(start_str, '%b-%d-%y')
    return start_date.strftime('%U')

getWkNum("Dec 30", 18)

'52'

In [16]:
def stripUSD(amount):
    return ''.join(amount.replace('$', '').split(','))

In [18]:
pds2018_cols = ['Year', 'BOM_id', 'Movie']
pds_rows = []
#pds_row = ['2018', res2018['ids'][0], res2018['titles'][0]]
for i in np.arange(53):
    lbl = i
    if i < 10:
        lbl = f'0{i}'
    pds2018_cols.append(f'W_{lbl}')
    #pds_row.append('0')
for i in np.arange(53):
    lbl = i
    if i < 10:
        lbl = f'0{i}'
    pds2018_cols.append(f'T_{lbl}')
    #pds_row.append('0')
# m2018_df = pd.DataFrame([pds_row], columns=pds2018_cols)
# m2018_df
#mem_pds = res2018['pds'][0]
# for ind, row in mem_pds.iterrows():
#     calWkNum = getWkNum(row['Date'], 18)
#     pds_row[3 + int(calWkNum)] = stripUSD(row['WeeklyGross'])
#     pds_row[56 + int(calWkNum)] = stripUSD(row['Gross-to-Date'])
    

In [19]:
# for each movie in the list
for i in np.arange(len(res2018['pds'])):
    # get the title and id to start the row
    new_row = ['2018', res2018['ids'][i], res2018['titles'][i]]
    # add in 0's 
    for j in np.arange(106):
        new_row.append('0')

    itr_pds = res2018['pds'][i]
    last_col = 56
    last_grs = 0
    for ind, row in itr_pds.iterrows():
        calWkNum = getWkNum(row['Date'], 18)
#         print(f"row {i}: {calWkNum} > {int(calWkNum)+3} = {row['WeeklyGross']} > {int(calWkNum)+56} = {row['Gross-to-Date']}")
        last_col = 56 + int(calWkNum)
        last_grs = stripUSD(row['Gross-to-Date'])
        new_row[3 + int(calWkNum)] = stripUSD(row['WeeklyGross'])
        new_row[last_col] = last_grs
    for j in np.arange(last_col+1, 109):
        new_row[j] = last_grs
    pds_rows.append(new_row)

In [20]:
m2018_df = pd.DataFrame(pds_rows, columns=pds2018_cols)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(m2018_df)

,Year,BOM_id,Movie,W_00,W_01,W_02,W_03,W_04,W_05,W_06,W_07,W_08,W_09,W_10,W_11,W_12,W_13,W_14,W_15,W_16,W_17,W_18,W_19,W_20,W_21,W_22,W_23,W_24,W_25,W_26,W_27,W_28,W_29,W_30,W_31,W_32,W_33,W_34,W_35,W_36,W_37,W_38,W_39,W_40,W_41,W_42,W_43,W_44,W_45,W_46,W_47,W_48,W_49,W_50,W_51,W_52,T_00,T_01,T_02,T_03,T_04,T_05,T_06,T_07,T_08,T_09,T_10,T_11,T_12,T_13,T_14,T_15,T_16,T_17,T_18,T_19,T_20,T_21,T_22,T_23,T_24,T_25,T_26,T_27,T_28,T_29,T_30,T_31,T_32,T_33,T_34,T_35,T_36,T_37,T_38,T_39,T_40,T_41,T_42,T_43,T_44,T_45,T_46,T_47,T_48,T_49,T_50,T_51,T_52
0,2018,marvel2017b,Black Panther,0,0,0,0,0,0,291954422,143445615,85479564,57496927,35881708,25178398,17489106,11529782,7986587,7186380,6352126,4273996,2707174,1174457,746707,368118,217287,203114,113230,60009,42286,19869,45572,16592,90540,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,291954422,435400037,520879601,578376528,614258236,639436634,656925740,668455522,676442109,683628489,689980615,694254611,696961785,698136242,698882949,699251067,699468354,699671468,699784698,699844707,699886993,699906862,699952434,699969026,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566
1,2018,theincredibles2,Incredibles 2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,269446690,124736824,81177900,44237078,26216948,19808208,12508642,8210310,5456248,3614118,2447947,5252592,1755520,955333,667224,353505,417506,276976,220915,180084,212891,166736,131124,66319,38406,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,269446690,394183514,475361414,519598492,545815440,565623648,578132290,586342600,591798848,595412966,597860913,603113505,604869025,605824358,606491582,606845087,607262593,607539569,607760484,607940568,608153459,608320195,608451319,608517638,608556044,608556044,608556044,608556044,608556044,608556044
2,2018,foxmarvel18,Deadpool 2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,164707352,66620086,33688103,20865267,13019513,7999537,5971542,2584220,1191820,751414,440675,285867,153215,86806,50105,43040,10320,13518,3376,3239,1660,751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,164707352,231327438,265015541,285880808,298900321,306899858,312871400,315455620,316647440,317398854,317839529,318125396,318278611,318365417,318415522,318458562,318468882,318482400,318485776,318489015,318490675,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426
3,2018,missionimpossible6,Mission: Impossible - Fallout,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89487371,52479913,28271437,15661939,11445869,10970238,5503570,3154330,1594266,863496,497901,228774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89487371,141967284,170238721,185900660,197346529,208316767,213820337,216974667,218568933,219432429,219930330,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104
4,2018,untitledhansolostarwarsanthologyfilm,Solo: A Star Wars Story,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,119592692,41358782,22812613,14367783,6854245,4873982,1652879,661331,404452,257742,317944,207217,159987,103581,91639,32885,17758,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,119592692,160951474,183764087,198131870,204986115,209860097,211512976,212174307,212578759,212836501,213154445,213361662,213521649,213625230,213716869,213749754,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512
5,2018,astarisborn2018,A Star is Born (2018),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,66160360,40915886,27501154,19957166,15475822,11480519,6496136,3930393,2671161,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0